In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:13 https://developer.download.nvidia.com/compute/c

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-07-03 03:32:55--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.04MB/s    in 0.2s    

2021-07-03 03:32:55 (6.04 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge-2").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   27288431| R33UPQQUZQEM8|B005T4ND06|     400024643|Yoga for Movement...|       Video DVD|          5|            3|          3|   N|                Y|This was a gift f...|This was a gift f...| 2015-08-31|
|         US|   13722556|R3IKTNQQPD9662|B004EPZ070|     685335564|  Something Borrowed|       Video DVD|          5|    

In [5]:
df2 = df.select(["star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
df2.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|            3|          3|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            1|          1|   N|                Y|
|          5|            0|          1|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          4|            0|          0|   N|                Y|
|          5|            1|          2|   N|                Y|
|          4|            1|          1|   N|                Y|
|          4|            1|          1|   N|           

In [6]:
df3 = df2.filter(df2["total_votes"] >= 20)
df3.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          4|           32|         39|   N|                N|
|          1|            4|         86|   N|                N|
|          4|           35|         37|   N|                N|
|          5|          139|        141|   N|                Y|
|          1|            0|         28|   N|                Y|
|          5|           16|         21|   N|                N|
|          1|            8|         23|   N|                N|
|          4|           95|        103|   N|                Y|
|          1|            9|         53|   N|                N|
|          5|           42|         50|   N|                N|
|          1|            1|         33|   N|                N|
|          1|           16|         20|   N|                N|
|          1|            9|         50|   N|           

In [7]:
df4 = df3.filter(df3["helpful_votes"]/df3["total_votes"]>=0.5)
df4.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          4|           32|         39|   N|                N|
|          4|           35|         37|   N|                N|
|          5|          139|        141|   N|                Y|
|          5|           16|         21|   N|                N|
|          4|           95|        103|   N|                Y|
|          5|           42|         50|   N|                N|
|          1|           16|         20|   N|                N|
|          5|           53|         55|   N|                Y|
|          4|           26|         28|   N|                N|
|          5|           30|         32|   N|                Y|
|          5|           26|         37|   N|                N|
|          5|           35|         38|   N|                Y|
|          4|           22|         24|   N|           

In [8]:
paid_df = df4.filter(df4["vine"] == 'Y')
paid_df.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          3|           66|         72|   Y|                N|
|          2|           20|         25|   Y|                N|
|          5|          236|        243|   Y|                N|
|          3|           11|         20|   Y|                N|
|          4|           16|         20|   Y|                N|
|          4|           27|         29|   Y|                N|
|          2|           27|         29|   Y|                N|
|          3|           37|         44|   Y|                N|
|          5|           21|         23|   Y|                N|
|          4|           29|         33|   Y|                N|
|          4|           18|         21|   Y|                N|
|          3|           18|         22|   Y|                N|
|          5|           32|         43|   Y|           

In [9]:
unpaid_df = df4.filter(df4["vine"] == 'N')
unpaid_df.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          4|           32|         39|   N|                N|
|          4|           35|         37|   N|                N|
|          5|          139|        141|   N|                Y|
|          5|           16|         21|   N|                N|
|          4|           95|        103|   N|                Y|
|          5|           42|         50|   N|                N|
|          1|           16|         20|   N|                N|
|          5|           53|         55|   N|                Y|
|          4|           26|         28|   N|                N|
|          5|           30|         32|   N|                Y|
|          5|           26|         37|   N|                N|
|          5|           35|         38|   N|                Y|
|          4|           22|         24|   N|           

In [10]:
paid_five_star_number = paid_df[paid_df['star_rating'] == 5].count()
print(paid_five_star_number)

9


In [11]:
paid_number = paid_df.count()
print(paid_number)

49


In [12]:
percent_five_star_vine = float(paid_five_star_number)/float(paid_number)
print(percent_five_star_vine)

0.1836734693877551


In [13]:
unpaid_five_star_number = unpaid_df[unpaid_df['star_rating'] == 5].count()
print(unpaid_five_star_number)

78061


In [14]:
unpaid_number = unpaid_df.count()
print(unpaid_number)

151400


In [16]:
percent_unpaid_five_star_vine = float(unpaid_five_star_number)/float(unpaid_number)
print(percent_unpaid_five_star_vine)

0.5155944517833554
